# Covasim

## Running a simulation

In [1]:
import covasim as cv

pars = dict(
  pop_type='hybrid',
  pop_size=10000
)

sim = cv.Sim(pars)

Covasim 2.0.2 (2020-02-01) — © 2021 by IDM


In [2]:
intervention_pars = [
 'beta',         # beta per symptomatic contact
 'contacts',     # the number of contacts per layer
 'dynam_layer',  # which layers are dynamic
 'beta_layer',   # transmissibility per layer
 'n_imports',    # average daily number of imported cases (actual number is drawn from Poisson distribution)
 'asymp_factor', # multiply beta by this factor for asymptomatic cases; no statistically significant difference in transmissibility
 'iso_factor',   # multiply beta by this factor for diganosed cases to represent isolation
 'quar_factor',  # quarantine multiplier on transmissibility and susceptibility
 'quar_period',  # number of days to quarantine for
 'interventions' # the interventions present in this simulation
]

for key in intervention_pars:
  print(f'{key:13} --> {sim[key]}')

beta          --> 0.016
contacts      --> {'h': 2.0, 's': 20, 'w': 16, 'c': 20}
dynam_layer   --> {'h': 0, 's': 0, 'w': 0, 'c': 0}
beta_layer    --> {'h': 3.0, 's': 0.6, 'w': 0.6, 'c': 0.3}
n_imports     --> 0
asymp_factor  --> 1.0
iso_factor    --> {'h': 0.3, 's': 0.1, 'w': 0.1, 'c': 0.1}
quar_factor   --> {'h': 0.6, 's': 0.2, 'w': 0.2, 'c': 0.2}
quar_period   --> 14
interventions --> []


In [3]:
sim.run()

Initializing sim with 10000 people for 60 days
  Running 2020-03-01 ( 0/60) (0.72 s)  ———————————————————— 2%
  Running 2020-03-11 (10/60) (0.82 s)  •••————————————————— 18%
  Running 2020-03-21 (20/60) (0.90 s)  ••••••—————————————— 34%
  Running 2020-03-31 (30/60) (0.98 s)  ••••••••••—————————— 51%
  Running 2020-04-10 (40/60) (1.06 s)  •••••••••••••——————— 67%
  Running 2020-04-20 (50/60) (1.14 s)  ••••••••••••••••———— 84%
  Running 2020-04-30 (60/60) (1.25 s)  •••••••••••••••••••• 100%

Simulation summary:
    6342 cumulative infections
    5377 cumulative infectious
       0 cumulative tests
       0 cumulative diagnoses
    3066 cumulative recoveries
    3494 cumulative symptomatic cases
     205 cumulative severe cases
      49 cumulative critical cases
       9 cumulative deaths
       0 cumulative quarantined people



## Running multiple simulations

In [5]:
multi_pars = [ (k, v) for k, v in dict(
  rand_seed = range(3),
  beta = [0.01, 0.02, 0.03]
).items()]

multi_pars

[('rand_seed', range(0, 3)), ('beta', [0.01, 0.02, 0.03])]

In [6]:
def cartesian_product(multiple_parameters: list, parameters: dict):
  if len(multiple_parameters) == 0:
    return [ parameters ]
  else:
    cart_product = []
    parameter, values = multiple_parameters[0]
    for value in values:
      new_parameters = {**parameters, parameter: value}
      sub_product = cartesian_product(multiple_parameters[1:], new_parameters)
      cart_product.append(sub_product)
    return [parameters for sub_product in cart_product for parameters in sub_product]

pars['verbose'] = 0
pars_list = cartesian_product(multi_pars, pars)
pars_list

[{'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 0,
  'beta': 0.01},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 0,
  'beta': 0.02},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 0,
  'beta': 0.03},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 1,
  'beta': 0.01},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 1,
  'beta': 0.02},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 1,
  'beta': 0.03},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 2,
  'beta': 0.01},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 2,
  'beta': 0.02},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 2,
  'beta': 0.03}]

In [7]:
sims = [cv.Sim(pars, label=f'Sim {i}') for i, pars in enumerate(pars_list)]
msim = cv.MultiSim(sims)
msim.brief()

MultiSim(n_sims: 9; base: Sim("Sim 0"; 2020-03-01 to 2020-04-30; pop: 10000 hybrid; epi: not run))


In [8]:
%%time
msim.run()

CPU times: user 88.8 ms, sys: 50.9 ms, total: 140 ms
Wall time: 4.58 s


In [9]:
keys = [key for key in msim.sims[0].summary.keys() if 'cum' in key]
values = []

for sim in msim.sims:
  values.append(sim.summary[keys])
values

[[1653.0, 1358.0, 0.0, 0.0, 917.0, 919.0, 53.0, 12.0, 2.0, 0.0],
 [8280.0, 7827.0, 0.0, 0.0, 5824.0, 5174.0, 354.0, 79.0, 24.0, 0.0],
 [9367.0, 9356.0, 0.0, 0.0, 9123.0, 6332.0, 524.0, 141.0, 55.0, 0.0],
 [1164.0, 945.0, 0.0, 0.0, 569.0, 621.0, 31.0, 9.0, 1.0, 0.0],
 [8266.0, 7686.0, 0.0, 0.0, 5449.0, 5103.0, 363.0, 89.0, 17.0, 0.0],
 [9376.0, 9348.0, 0.0, 0.0, 8990.0, 6306.0, 520.0, 146.0, 60.0, 0.0],
 [2908.0, 2414.0, 0.0, 0.0, 1518.0, 1580.0, 83.0, 20.0, 4.0, 0.0],
 [8810.0, 8664.0, 0.0, 0.0, 7650.0, 5846.0, 433.0, 108.0, 35.0, 0.0],
 [9493.0, 9487.0, 0.0, 0.0, 9282.0, 6363.0, 505.0, 149.0, 46.0, 0.0]]